In [1]:
import numpy as np
import random

# 环境设置：5x5 网格世界
class GridWorld:
    def __init__(self):
        self.grid_size = 5
        self.start = (0, 0)  # 起点
        self.goal = (4, 4)   # 目标
        self.actions = ['up', 'down', 'left', 'right']  # 动作空间
        self.action_to_index = {a: i for i, a in enumerate(self.actions)}
        self.state = self.start  # 当前状态

    def reset(self):
        self.state = self.start
        return self.state

    def step(self, action):
        x, y = self.state
        # 根据动作更新位置
        if action == 'up':
            x = max(x - 1, 0)
        elif action == 'down':
            x = min(x + 1, self.grid_size - 1)
        elif action == 'left':
            y = max(y - 1, 0)
        elif action == 'right':
            y = min(y + 1, self.grid_size - 1)

        next_state = (x, y)
        # 计算奖励
        if next_state == self.goal:
            reward = 100  # 到达目标
            done = True
        else:
            reward = -1  # 每移动一步 -1
            if next_state == self.state:  # 撞墙
                reward = -10
            done = False

        self.state = next_state
        return next_state, reward, done

    def get_action_space(self):
        return self.actions

In [2]:
# SARSA 算法
class SARSAAgent:
    def __init__(self, grid_size, actions, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.grid_size = grid_size
        self.actions = actions
        self.action_to_index = {a: i for i, a in enumerate(actions)}
        self.alpha = alpha  # 学习率
        self.gamma = gamma  # 折扣因子
        self.epsilon = epsilon  # 探索率
        # 初始化 Q 表：(状态, 动作) -> Q 值
        self.q_table = np.zeros((grid_size, grid_size, len(actions)))
        
    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:  # 探索
            return random.choice(self.actions)
        else:  # 利用
            x, y = state
            action_idx = np.argmax(self.q_table[x, y])
            return self.actions[action_idx]
        
    
    def update(self, state, action, reward, next_state, next_action):
        x, y = state
        nx, ny = next_state
        action_idx = self.action_to_index[action]
        next_action_idx = self.action_to_index[next_action]
        # SARSA 更新公式
        self.q_table[x, y, action_idx] += self.alpha * (
            reward + self.gamma * self.q_table[nx, ny, next_action_idx] - self.q_table[x, y, action_idx]
        )

    def get_policy(self, state):
        x, y = state
        action_idx = np.argmax(self.q_table[x, y])
        return self.actions[action_idx]

In [3]:
# 训练和测试
def train():
    env = GridWorld()
    agent = SARSAAgent(env.grid_size, env.get_action_space())

    # 训练参数
    num_episodes = 1000
    max_steps = 100

    for episode in range(num_episodes):
        state = env.reset()
        action = agent.choose_action(state)  # 初始动作
        for step in range(max_steps):
            next_state, reward, done = env.step(action)
            next_action = agent.choose_action(next_state)  # SARSA 的在策略特性
            agent.update(state, action, reward, next_state, next_action)
            state = next_state
            action = next_action
            if done:
                break

    # 测试：打印最优路径
    print("训练完成！以下是从起点到目标的最优路径：")
    state = env.reset()
    path = [state]
    steps = 0
    while state != env.goal and steps < max_steps:
        action = agent.get_policy(state)
        state, _, done = env.step(action)
        path.append(state)
        steps += 1

    for s in path:
        print(s, end=" -> ")
    print("到达目标！")

if __name__ == "__main__":
    train()

训练完成！以下是从起点到目标的最优路径：
(0, 0) -> (0, 1) -> (1, 1) -> (2, 1) -> (3, 1) -> (3, 2) -> (3, 3) -> (4, 3) -> (4, 4) -> 到达目标！
